In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 10})

In [ ]:
line_dict = {
    r'[OIII]$_{\lambda 5008}$': 5008,
    r'[OIII]$_{\lambda 4960}$': 4960,
    r'H$\beta$': 4862,
    r'[OIII]$_{\lambda 4364}$': 4364,
    r'H$\gamma$': 4341
}

line_dict_items = [[a, b] for a, b in line_dict.items()]

In [ ]:
FIELD = 'COLA1'
thisID = 9269

this_z = 6.5916

fig, axes = plt.subplots(5, 2, figsize=(5, 12), sharey='row')

for j in range(axes.shape[0]):
    thisline = line_dict_items[j][1]

    flux_tot_mods = np.array([np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_modA_fluxtot.npy'),
                     np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_modB_fluxtot.npy')])
    flux_tot_err_mods = np.array([np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_modA_fluxtoterr.npy'),
                     np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_modB_fluxtoterr.npy')])
    
    mask_both_zeros = ~np.all(flux_tot_mods == 0, axis=0)
    flux_tot_AB = np.zeros_like(flux_tot_mods[0])
    flux_tot_AB[mask_both_zeros] = np.average(np.array(flux_tot_mods)[:, mask_both_zeros],
                             axis=0, weights=np.array(flux_tot_err_mods)[:, mask_both_zeros]**-2)

    for i, mod in enumerate(['A', 'B']):
        this_ax = axes.T[i, j]

        flux_tot = flux_tot_mods[i]
        flux_tot_err = flux_tot_err_mods[i]

        obs_wav = np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_mod{mod}_obswav.npy')
        xx = np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_mod{mod}_xx.npy')
        yy = np.load(f'../COLA1_fluxes/{FIELD}_{thisID}_{int(thisline)}_mod{mod}_yy.npy')

        wav_0_min = thisline - 16
        wav_0_max = thisline + 16
        sel_wav = (obs_wav > wav_0_min * (1 + this_z) - 20) & (obs_wav < wav_0_max * (1 + this_z) + 20)

        this_ax.plot(obs_wav[sel_wav], flux_tot[sel_wav], drawstyle='steps-mid', c='k',
                     lw=1.5)
        this_ax.plot(obs_wav[sel_wav], flux_tot_AB[sel_wav], c='r', alpha=0.7,
                     lw=1.5, zorder=-99, ls='--', drawstyle='steps-mid')
        this_ax.plot(xx, yy, lw=1.5, c='cornflowerblue')
        this_ax.fill_between(obs_wav[sel_wav], -flux_tot_err[sel_wav],
                        flux_tot_err[sel_wav], lw=0, alpha=0.4, color='cornflowerblue')

        # Plot params
        this_ax.tick_params(direction='in', which='both')
        this_ax.yaxis.set_ticks_position('both')
        this_ax.xaxis.set_ticks_position('both')

        if [i, j] == [0, 2]:
            this_ax.set_ylabel(r'Flux [$10^{18}$\,erg\,s$^{-1}$\,cm$^{-2}$\,\AA$^{-1}$]',
                               fontsize=15)
        if j == 4:
            this_ax.set_xlabel(r'Wavelength [nm]',
                               fontsize=15)

        xlim_low = wav_0_min * (1 + this_z)
        xlim_high = wav_0_max * (1 + this_z)
        this_ax.set_xlim(xlim_low, xlim_high)

        # xticks
        xticks = np.arange(32850, 38200, 50)
        xticks = xticks[((xticks > xlim_low + 0)
                         & (xticks < xlim_high - 0))]
        this_ax.set_xticks(xticks)
        this_ax.set_xticklabels([f'{xt/10:0.0f}' for xt in xticks])

        # Make legend with dummy point to indicate the line
        line_name = line_dict_items[j][0]
        this_ax.plot([], [], ls='', label=line_name)
        this_ax.legend(fontsize=10, loc='upper left', markerfirst=False,
                       frameon=False)

        # Title "modA" or "modB"
        if j == 0 and i == 0:
            this_ax.set_title(r'{ \bf Module A}', fontsize=12)
        elif j == 0 and i == 1:
            this_ax.set_title(r'{ \bf Module B}', fontsize=12)



fig.subplots_adjust(wspace=0.02, hspace=0.15)
savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/COLA1_all_line_fits.pdf', bbox_inches='tight', pad_inches=0.01,
            facecolor='w')
plt.show()